In [295]:
import os
import re
import spacy
import pandas as pd

# Preprocess Text

In [328]:
FILE_PATH = "../assets/NER_assets/Ancient_Words.csv"
# read csv file
df = pd.read_csv(FILE_PATH)

In [329]:
# rename columns to fit code
df.rename(columns = {'Word':'Keyword', 'Category Types':'Label'}, inplace = True)
# If a cell is empty (NaN), Fill it with the value in its parallel "Early" column
for row in df:
    df['Quote'].fillna(df['Early Quote'], inplace=True)
    df['Word Before'].fillna(df['Early Word Before'], inplace=True)
    df['Word After'].fillna(df['Early Word After'], inplace=True)
    df['Label'].fillna(df['Early Category Type'], inplace=True)
# remove rows with no Keyword
df = df.dropna(subset=['Keyword'])
# Remove any row that isn't Greek
pat = '[ء-ي]+'
#df.Keyword.str.contains(pat)
df = df[~df.Keyword.str.contains(pat, na=False)]
#replace new line in df column
df['Keyword'].replace('\n', '', regex=True, inplace=True)
#replace numbers in df
df.replace('\d+', '', regex=True, inplace=True)
#replace hyphens in df column
df.replace('-', '', regex=True, inplace=True)
# replace comma in df column
df['Keyword'].replace(',', '', regex=True, inplace=True)
#replace period in df column
df['Keyword'].replace('\.', '', regex=True, inplace=True)
#replace interpunkt in df column
df['Keyword'].replace('\·', '', regex=True, inplace=True)
# replace multiple spaces in df column
df.replace(' +', ' ', regex=True, inplace=True)
# replace end punctuation in df column
df['Keyword'].replace('\s+$', '', regex=True, inplace=True)

df.fillna(0)
df.reset_index(drop=True, inplace=True)


In [330]:
df.head(10)

,Keyword,Word Before,Word After,Quote,Label,Lemma,Early Category Type,Early Word,Early Word Before,Early Word After,Early Quote,Lemma arabic
0,οὖλον,δὲ πολυφυὲς,· σάρκινα δὲ,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Body Part,οὖλον,Body Part,οὖλον,δὲ πολυφυὲς,· σάρκινα δὲ,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
1,παρίσθμιον,τοῦ στόματος,", τὸ δὲ",Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Body Part,παρίσθμιον,Body Part,παρίσθμιον,τοῦ στόματος,", τὸ δὲ",Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
2,πολυφυὲς,τὸ δὲ,οὖλον· σάρκινα,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Adjectives/Qualities,πολυφυής,Adjectives/Qualities,πολυφυὲς,τὸ δὲ,οὖλον· σάρκινα,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
3,μόριον,δ’ ἄλλο,"σταφυλοφόρον, κίων","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",Body Part,μόριον,Body Part,μόριον,δ’ ἄλλο,"σταφυλοφόρον, κίων","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",NaN
4,ὀδόντες,Ἐντὸς δ’,ὀστέινοι. Εἴσω,Ἐντὸς δ’ ὀδόντες ὀστέινοι,Body Part,ὀδούς,Body Part,ὀδόντες,Ἐντὸς δ’,ὀστέινοι. Εἴσω,Ἐντὸς δ’ ὀδόντες ὀστέινοι,NaN
5,ὀστέινοι,δ’ ὀδόντες,. Εἴσω δ’,Ἐντὸς δ’ ὀδόντες ὀστέινοι,Adjectives/Qualities,ὀστέινος,Adjectives/Qualities,ὀστέινοι,δ’ ὀδόντες,. Εἴσω δ’,Ἐντὸς δ’ ὀδόντες ὀστέινοι,NaN
6,σταφυλοφόρον,ἄλλο μόριον,", κίων ἐπίφλεβος·","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",Adjectives/Qualities,σταφυλοφόρος,Adjectives/Qualities,σταφυλοφόρον,ἄλλο μόριον,", κίων ἐπίφλεβος·","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",NaN
7,ἐπίφλεβος,"σταφυλοφόρον, κίων",· ὃς ἐὰν,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",Adjectives/Qualities,ἐπίφλεβος,Adjectives/Qualities,ἐπίφλεβος,"σταφυλοφόρον, κίων",· ὃς ἐὰν,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",NaN
8,κίων,"μόριον σταφυλοφόρον,",ἐπίφλεβος· ὃς,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",Adjectives/Qualities,κίων,Adjectives/Qualities,κίων,"μόριον σταφυλοφόρον,",ἐπίφλεβος· ὃς,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",NaN
9,μέρος,Ἔτι προσώπου,τὸ μὲν,Ἔτι προσώπου μέρος τὸ μὲν ὂν τῷ πνευ...,Adjectives/Qualities,μέρος,Adjectives/Qualities,μέρος,Ἔτι προσώπου,τὸ μὲν,Ἔτι προσώπου μέρος τὸ μὲν ὂν τῷ πνευ...,NaN


In [331]:
# if any of the fields "KeyWord", "Quote", "Word Before", "Word After" are "0", drop the row
for w in ['Keyword', 'Quote', 'Word Before', 'Word After']:
    df = df[df[w] != 0]


In [332]:
# import requirements for converting the dataframe to Spacy Docs
from collections import defaultdict
from typing import List
from spacy.language import Language
from spacy.tokens import Doc, DocBin, Span
from spacy.util import filter_spans
from unicodedata import normalize
import regex
import random


In [333]:
d = {ord('\N{COMBINING ACUTE ACCENT}'):None, ord('\N{COMBINING COMMA ABOVE}'):None, ord('\N{COMBINING REVERSED COMMA ABOVE}'):None}

def find_word_index(sentence, word, word_before, word_after):
    """Find the index of a word in a sentence, it can appear multiple times but we return by word_before and word_after"""


    # find the index of the word in the sentence (return index of beginning of word)
    word_index = sentence.find(word)

    # return all chars of word before: [),.,·] (meaning none of these chars will be included) so we get only data contained within the sentence
    for char in [")", ".", "·"]:
        if word_before.find(char) != -1:
            word_before = word_before[word_before.find(char)+1:]
        if word_after.find(char) != -1:
            word_after = word_after[:word_after.find(char)]



    pattern = fr"{re.escape(word_before)}({(word)}){re.escape(word_after)}"
    # add fuzzy matching with up to 3 mistakes
    pattern = fr"(?:{pattern}){{e<=3}}"



    match = regex.search(pattern, sentence)
    try:
        print ("match:", match.span(1))
        return match.span(1)
    except:

        return None



In [334]:
# normalize table
FORMAT = 'NFD'
for col in ['Keyword', 'Quote', 'Word Before', 'Word After']:
    df[col] = df[col].apply(lambda x: normalize(FORMAT, x))

In [353]:
# using word before and word after, we want to bring our data into the following format:
# TRAIN_DATA = [ (TEXT AS A STRING, {“entities”: [(START, END, LABEL)]}) ]

TRAIN_DATA = []
train_data_df = pd.DataFrame(columns=['text', 'keyword', 'index', 'label', 'lemma'])
no_found = 0

for i,row in df.iterrows():
    # fill train_data_df with index from find_word_index
    index = find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After'])
    if index:
        row_to_add = [row['Quote'], row['Keyword'], index, row['Label'], row['Lemma']]
        train_data_df = pd.concat([train_data_df, pd.DataFrame([row_to_add], columns=['text', 'keyword', 'index', 'label', 'lemma'])], ignore_index=True)
    else:
        print ("No index found for", row['Keyword'], "in", row['Quote'])
        no_found += 1

print ("No index found for", no_found, "words")

match: (68, 76)
match: (36, 48)
match: (58, 68)
match: (16, 24)
match: (10, 20)
match: (20, 31)
match: (24, 38)
match: (45, 57)
match: (39, 45)
match: (15, 22)
match: (41, 51)
match: (51, 58)
match: (5, 15)
match: (58, 64)
match: (9, 19)
match: (60, 65)
match: (24, 33)
match: (51, 60)
match: (33, 41)
No index found for ἀθρόου in καὶ γὰρ ἀναπνεῖ καὶ ἐκπνεῖ ταύτῃ, καὶ ὁ πταρμὸς διὰ ταύτης γίνεται , πνεύματος ἀθρόου ἔξοδος , σημεῖον οἰωνιστικὸν καὶ ἱερὸν μόνον τῶν πνευμάτων
match: (72, 81)
match: (103, 112)
match: (11, 24)
match: (41, 50)
match: (50, 55)
match: (29, 41)
match: (59, 67)
match: (92, 98)
match: (85, 92)
match: (108, 121)
match: (139, 144)
match: (148, 152)
match: (125, 137)
match: (97, 108)
match: (157, 166)
match: (178, 189)
match: (215, 227)
match: (194, 204)
match: (204, 210)
match: (237, 241)
match: (246, 255)
match: (16, 25)
match: (25, 30)
match: (42, 47)
match: (43, 51)
match: (5, 16)
match: (5, 16)
match: (20, 31)
match: (0, 11)
match: 

Please notice the above warning about using a deprecated method, and change accordingly.\
<code>FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.</code>


In [345]:
# compare text[index] with keyword
test_data_df = pd.DataFrame(columns=['text', 'real_keyword', 'function_found'])
# count nulls in train_data_df
nulls = train_data_df.isnull().sum()
# drop nulls in train_data_df
train_data_df = train_data_df.dropna()
# iterate over all rows in train_data_df
for index, row in train_data_df.iterrows():
    test_data_df = test_data_df.append({'text': row['text'], 'real_keyword': row['keyword'], 'function_found': row['text'][row['index'][0]:row['index'][1]]}, ignore_index=True)


In [346]:
# print nulls in train_data_df
print ("Nulls", nulls)
# print shape of train_data_df
print ("\nShape", train_data_df.shape)


Nulls text       0
keyword    0
index      0
label      2
lemma      5
dtype: int64

Shape (1476, 5)


In [347]:
# sample test_data_df where real_keyword != function_found
test_data_df[test_data_df['real_keyword'] != test_data_df['function_found']].sample(10)

,text,real_keyword,function_found
1068,πλὴν οὐκ εἰς τὸ κοῖλον ἀλλ’ εἰς τὸ σῶ...,νεφρῶν,νεφρῶν
1407,διατέταται δὲ καὶ ἀπὸ τοῦ σπληνὸς φλὲψ...,ἀπὸ,ἀπὸ
863,Ἕτερον δὲ μέρος ἀπὸ τῶν ἀριστερῶν τῆ...,ἀναβαίνει,ἀναβαίνει
457,Ἀλλ’ οἱ νομίζοντες εἶναι κενὸν διηπάτην...,ἐκ,ἐκ
60,ἔστι γὰρ ὁ μυκτὴρ διχότομος,διχότομος,διχότομος
571,"διὰ μὲν γὰρ τῶν ἄλλων σπλάγχνων, ᾗ τ...",κάτω,κάτω
780,Τὸν αὐτὸν δὲ τρόπον καὶ τὰ τῆς ἐλάττ...,ἀορτῆς,ἀορτῆς
52,"Εὐκίνητος δ’ ὁ μυκτήρ, καὶ οὐχ ὥσπερ τ...",οὖς,οὖς
508,εἰσὶ δὲ καὶ τῶν περὶ φύσιν οἳ τοιαύτ...,ἀρχὴν,ἀρχὴν
768,Τῶν δὲ λοιπῶν τῶν ἀπὸ τῆς φλεβὸς ταύτ...,αἱ,αἱ


# Create dictionaries from dendrosearch and conllu files (supplied by Jackobo)

In [348]:
PUNCTUATION = ['.', ")", ".", "·", "(", "[", "]", ":", ";", ",", "?", "!", "،", "_"]
# extract from df a dictionary {word: lemma}
lemma_dict = {}
for index, row in df.iterrows():
    lemma_dict[row['Keyword']] = row['Lemma']

# load dendrosearch lemma dictionary
dendrosearch_lemma_dict = {}
with open('../assets/dendrosearch_lemma_dict.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.split()
        # check if not punctuation
        if len(line) > 1 and line[0] not in PUNCTUATION:
            dendrosearch_lemma_dict[line[0]] = line[1]

# create dictionary from all conllu files
PATH = "../assets/Lemmatization_training_files/"
conllu_lemma_dict = {}

# iterate over all files in directory
for f in os.listdir(PATH):
    if f.endswith(".conllu"):
        # if file is a conllu file
        with open(os.path.join(PATH, f), 'r', encoding='utf-8') as f:
            for line in f:
                # conll line is: id | keyword | lemma | pos | _
                # we want only keyword and lemma
                line = line.split()
                if len(line) > 2 and line[1] not in PUNCTUATION:
                    conllu_lemma_dict[line[1]] = line[2]



1. Error fix: I added above the line:\
<code>if f.endswith(".conllu")</code>\
Without it it gets an error because it tries to read folders and other files as well.\ :)
2. Why do you add the conllu to dictionary? ANSWER: I thought they might be useful for lemmatization, anyways they can't hurt, and if you don't want them you can just ignore them.

## Create dictionary from INCEpTION files

In [305]:
from cassis import *
import zipfile
import tempfile

inception_dict = {}
inception_sentences = []

In [306]:
# extract all files in inception folder to temp folder
with tempfile.TemporaryDirectory() as tempdir:
    for f in os.listdir("../assets/NER_assets/INCEpTION_files/"):
        if f.endswith(".zip"):
            with zipfile.ZipFile(os.path.join("../assets/NER_assets/INCEpTION_files/", f), 'r') as zip_ref:
                zip_ref.extractall(tempdir)
    print (tempdir)
    # open typesystem and print content
    with open('{0}/{1}'.format(tempdir, "TypeSystem.xml"), 'rb') as f:
        typesystem = load_typesystem(f)

    # iterate over all files in temp folder
    for f in os.listdir(tempdir):
        # if file is a xmi file
        print (typesystem)
        if f.endswith(".xmi"):
            # load xmi file
            with open(os.path.join(tempdir, f), 'rb') as f:
                # load typesystem from temp folder
                cas = load_cas_from_xmi(f, typesystem=typesystem)
                for token in cas.select('de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Lemma'):
                    inception_dict[token.get_covered_text()] = token.value
                for sentence in cas.select("de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence"):
                    inception_sentences.append(sentence.get_covered_text())


C:\Users\roeyz\AppData\Local\Temp\tmp_c9l23fr


In [307]:
# add to big dictionary and sample
big_dict = {**lemma_dict, **dendrosearch_lemma_dict, **conllu_lemma_dict, **inception_dict}

random.sample(list(big_dict.items()), 10)

[('ἐρείπει', 'ἐρείπω'),
 ('διαλεχθέντος', 'διαλέγω'),
 ('Ἀργὼ', 'Ἀργώ'),
 ('πρόες', 'προΐημι'),
 ('μνήστορες', 'μνήστωρ'),
 ('κουροτρόφος', 'κουροτρόφος'),
 ('ὀλιγόκαρπος', 'ὀλιγόκαρπος'),
 ('διέβην', 'διαβαίνω'),
 ('ἐπέστρεψα', 'ἐπιστρέφω'),
 ('παραβαίνεις', 'παραβαίνω')]

In [315]:
# create two copies of dictionary, one in NFD, other in NFKC
FIRST_FORMAT = 'NFD'
SECOND_FORMAT = 'NFKC'

big_dict_nfd = {}
big_dict_nfkc = {}

# remove all nan values from big_dict
big_dict = {k: v for k, v in big_dict.items() if pd.notnull(v)}

for key, value in big_dict.items():
    big_dict_nfd[normalize(FIRST_FORMAT, key)] = normalize(FIRST_FORMAT, value)
    big_dict_nfkc[normalize(SECOND_FORMAT, key)] = normalize(SECOND_FORMAT, value)

1. Notice the error above: TypeError: normalize() argument 2 must be str, not float
This is probably due to havving numbers as values in dictionary. NOTE: this in due to nan values in dictionary, anyways there are only 6 of them so I just removed them.
2. Why NFD and not NFKD? No reason , you can use NFKD if you want.

In [316]:
# check delta between two dictionaries
delta = {k: big_dict_nfd[k] for k in set(big_dict_nfd) - set(big_dict_nfkc)}
# delta size estimation
print ("delta size:", len(delta))

delta size: 168984


# Run NLP pipeline on INCEpTION and Coda files

In [317]:
# merge sentences from inception and coda

sentences = df['Quote'].tolist() + inception_sentences
sentences = list(set(sentences))
random.sample(sentences, 10)

['Ἐντὸς δὲ τοῦ αὐχένος ὅ τε οἰσοφάγος καλούμενός ἐστιν, ἔχων τὴν ἐπωνυμίαν ἀπὸ τοῦ μήκους καὶ τῆς στενότητος, καὶ ἡ ἀρτηρία.',
 'Ἐπὶ δὲ θάτερα καθήκει εἰς τὸ μεταξὺ τοῦ πλεύμονος, εἶτ’ ἀπὸ τούτου σχίζεται εἰς ἑκάτερον τῶν μερῶν τοῦ πλεύμονος',
 'ἔστι γὰρ ὁ μυκτὴρ διχότομος ',
 'ῥάχιος \r\nεἴκοσι·',
 'Συνήρτηται δὲ καὶ ἡ καρδία τῇ ἀρτηρίᾳ πιμελώδεσι καὶ χονδρώδεσι καὶ ἰνώδεσι δεσμοῖς ',
 'καὶ γὰρ ἀναπνεῖ καὶ ἐκπνεῖ ταύτῃ, καὶ ὁ πταρμὸς διὰ ταύτης γίνεται , πνεύματος ἀθρόου ἔξοδος , σημεῖον οἰωνιστικὸν καὶ ἱερὸν μόνον τῶν πνευμάτων',
 'κάτω δὲ πρὸς τὴν χεῖρα τὸ ὀστέον ἄρθρον ἔχει·',
 'Δύο φλέβες εἰσὶν ἐν τῷ θώρακι κατὰ τὴν ῥάχιν μέν, ἐντός δὲ κείμεναι ταύτης, ἡ μὲν μείζων ἐν τοῖς πρόσθεν, ἡ δ’ ἐλάττων ὄπισθεν ταύτης, καὶ ἡ μὲν μείζων ἐν τοῖς δεξιοῖς μᾶλλον, ἡ δ’ ἐλάττων ἐν τοῖς ἀριστεροῖς, ἣν καλοῦσί τινες ἀορτὴν ἐκ τοῦ τεθεᾶσθαι κα

In [326]:
# run nlp pipeline on sentences
# install spacy grc model if not already installed
nlp = spacy.load("grc_proiel_sm") # I use small model for speed but you should use trf (transformer) model for better accuracy

# create list of Doc objects
docs: List[Doc] = []

# create df to record corrections
corrections_df = pd.DataFrame(columns=['sentence', 'token', 'lemma', 'lemma_corrected', 'dictionary'])
corrected_sentences = 0
for sentence in sentences:
    doc = nlp(sentence)
    # we use our dictionary to replace lemmas which the model didn't recognize correctly
    for token in doc:
        # if token text is a lexical value in our dictionary
        if token.text in big_dict_nfd:
            if token.lemma_ != big_dict_nfd[token.text]:
                corrections_df = pd.concat([corrections_df, pd.DataFrame({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfd[token.text], 'dictionary': 'big_dict_nfd'}, index=[0])], ignore_index=True)
                token.lemma_ = big_dict_nfd[token.text]

        elif token.text in big_dict_nfkc:
            if token.lemma_ != big_dict_nfkc[token.text]:
                corrections_df = pd.concat([corrections_df, pd.DataFrame({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfkc[token.text], 'dictionary': 'big_dict_nfkc'}, index=[0])], ignore_index=True)
                token.lemma_ = big_dict_nfkc[token.text]
    docs.append(doc)

#pandas append is deprecated, instead use concat

Again, the deprecation warning; I don't get these deprecation warnings.

In [319]:
print ("corrections_df size:", corrections_df.shape)

corrections_df size: (4525, 5)


In [320]:
corrections_df.sample(10)


,sentence,token,lemma,lemma_corrected,dictionary
1127,διότι οὐκ αἰεὶ κατὰ τωὐτὸ τῆς κεφαλὴς...,κεφαλὴς,κεφαλὴς,κεφαλή,big_dict_nfd
4356,Καὶ ἔστιν ἡ μὲν μεγάλη φλὲψ ὑμενώδης ...,ἔστιν,ἔστις,εἰμί,big_dict_nfd
4478,Τείνει δ’ ἀπ’ αὐτῆς μία μὲν διὰ τοῦ ἡ...,αὐτῆς,αὐτῆς,αὐτός,big_dict_nfd
277,Τὸ δὲ μετὰ τοῦτο εὐθὺ πρὸς τὴν ἔξοδο...,τοῖς,τοῖς,ὁ,big_dict_nfd
2673,Ὑπὸ δὲ τὸν πνεύμονά ἐστι τὸ διάζωμα τ...,ῥάχιν,ῥάχιν,ῥάχις,big_dict_nfd
2796,{ΠΕΡΙ ΑΝΑΤΟΜΗΣ.},ΠΕΡΙ,ΠΕΡΙ,_,big_dict_nfd
2604,οἷσι δ ἂν βόρειόν τε καὶ πάνυ ὀλίγον...,καὶ,καὶ,καί,big_dict_nfd
705,ἀλλ’ ἐν μὲν τοῖς ζῳοτόκοις οὐχ ὁμοίως...,τοῖς,τοῖς,ὁ,big_dict_nfd
3902,Ἔστι δ’ ἡ μὲν ἀρτηρία χονδρώδης τὴν φυ...,τοῦ,τοῦ,ὁ,big_dict_nfd
842,Ἅμα δ’ ἡ ἀνάπνευσις καὶ ἔκπνευσις γίν...,τοῖς,τοῖς,ὁ,big_dict_nfd


In [321]:
# find how many corrected by each dictionary
corrections_df.groupby('dictionary').count()


,sentence,token,lemma,lemma_corrected
dictionary,,,,
big_dict_nfd,4253,4253,4253,4253
big_dict_nfkc,272,272,272,272


In [322]:
corrections_df[corrections_df['dictionary'] == 'big_dict_nfkc'].sample(10)

,sentence,token,lemma,lemma_corrected,dictionary
234,"αἱ δὲ κάτω τοῦ σώματος \r\nτετραμμέναι, δύο μὲ...",σπονδύλους,σπονδύλος,σφόνδυλος,big_dict_nfkc
970,αὗται \r\nκαὶ ἄκαρπον ποιέουσι τὸν ἄνθρωπον ὅτ...,αἳ,ὅς,ὁ,big_dict_nfkc
231,"αἱ δὲ κάτω τοῦ σώματος \r\nτετραμμέναι, δύο μὲ...",δύο,γε,δύο,big_dict_nfkc
947,τὸ δ' ἀποτρεπόμενον ἀποσυμβολέει τῷ ἐπιρρέοντι·,τῷ,τῷ,ὁ,big_dict_nfkc
2507,καὶ τὸ \r\nμὲν ἀποτρεπόμενον βουλόμενον ἄνω χω...,ἀτρεκέως,ἀτρεκής,ἀτρεκέως,big_dict_nfkc
1450,"Νεφροὶ δὲ ὁμοιορυσμοὶ, \nτὴν χροιὴν δὲ ἐναλίγκ...",ἀπὸ,ἀπό,ἀπὸ,big_dict_nfkc
3501,Ἡπατῖτις ἐν ὀσφύϊ μέχρι τοῦ μεγάλου σπονδύλου ...,ἥπατος,ἥπατος,ἧπαρ,big_dict_nfkc
2610,"αἱ μὲν γὰρ σφίσιν \r\nἑωυτῇσι συμβάλλουσιν, αἱ...",ἀπὸ,ἀπό,ἀπὸ,big_dict_nfkc
3503,Ἡπατῖτις ἐν ὀσφύϊ μέχρι τοῦ μεγάλου σπονδύλου ...,φρενῶν,φρενόω,φρήν,big_dict_nfkc
1084,Τὸν αὐτὸν δὲ τρόπον καὶ ἐν τῷ θήλει πάντα πέφυ...,θεωρείσθω,θεωρείσθω,θεωρέω,big_dict_nfkc


In [323]:
corrections_df[corrections_df['dictionary'] == 'big_dict_nfd'].sample(10)

,sentence,token,lemma,lemma_corrected,dictionary
1278,τῶν μὲν γὰρ οἱ πόροι συγκεχυμένοι καθάπ...,ὀχετοί,ὀχετοί,ὀχετός,big_dict_nfd
1052,Ἀλλ’ οἱ νομίζοντες εἶναι κενὸν διηπάτην...,ἐξελήλυθε,ἐξελήλυθε,ἐξέρχομαι,big_dict_nfd
4223,Ἐκ μέσου δὲ τῶν νεφρῶν ἑκατέρου φλὲψ κ...,ἐξήρτηται,ἐξήρτομαι,ἐξαρτάω,big_dict_nfd
793,Καθάπτουσι δὲ πρὸς τὴν κύστιν καὶ τὸ αι...,τὰς,τὰς,ὁ,big_dict_nfd
3693,Ἅμα δ’ ἡ ἀνάπνευσις καὶ ἔκπνευσις γίν...,εἶναι,εἶναι,εἰμί,big_dict_nfd
2200,Ἡ μὲν οὖν μεγάλη φλὲψ ἐκ τῆς μεγίστης...,γίνεται,γίνομαι,γίγνομαι,big_dict_nfd
1668,Τὸ δ' ἐπίπλοον ἀπὸ μέσης τῆς κοιλίας ...,τοῖς,τοῖς,ὁ,big_dict_nfd
1150,Ἐπὶ δὲ θάτερα καθήκει εἰς τὸ μεταξὺ το...,μεταξὺ,μεταξὺ,μεταξύ,big_dict_nfd
1700,Πάλιν δ’ ἐντεῦθεν εἰς τέτταρας σχίζονται...,τοῦ,τοῦ,ὁ,big_dict_nfd
2408,Τῶν δὲ λοιπῶν τῶν ἀπὸ τῆς φλεβὸς ταύτ...,μὲν,μὲν,μέν,big_dict_nfd


In [351]:
# split docs to train, dev, test randomly
from sklearn.model_selection import train_test_split
from pathlib import Path


train_docs, test_docs = train_test_split(docs, test_size=0.2, random_state=42)
train_docs, dev_docs = train_test_split(train_docs, test_size=0.4, random_state=42)

print ("train_docs size:", len(train_docs))
print ("dev_docs size:", len(dev_docs))
print ("test_docs size:", len(test_docs))

# save each one to DocBin

Path("../corpus/train").mkdir(parents=True, exist_ok=True)
Path("../corpus/dev").mkdir(parents=True, exist_ok=True)
Path("../corpus/test").mkdir(parents=True, exist_ok=True)


train_bin = DocBin(docs=train_docs)
train_bin.to_disk('../corpus/train/lemma_train.spacy')
test_bin = DocBin(docs=test_docs)
test_bin.to_disk('../corpus/test/lemma_test.spacy')
dev_bin = DocBin(docs=dev_docs)
dev_bin.to_disk('../corpus/dev/lemma_dev.spacy')




train_docs size: 739
dev_docs size: 493
test_docs size: 308


The spacy dataset should be exported to '../corpus/' folder.\
More specifically:\
train to '..corpus/train/lemma_train/'\
dev to '../corpus/dev/lemma_dev/'\
test to '../corpus/test/lemma_test/